# Street View House Numbers Classification


In [0]:
import random
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.models import Sequential
from keras.layers import *
from keras.utils import to_categorical
from keras.constraints import max_norm
import sklearn.model_selection
from keras.preprocessing.image import ImageDataGenerator

## Loading All Three Sets

In [4]:
train_mat = keras.utils.get_file("train_32x32.mat", "https://s3-ap-southeast-1.amazonaws.com/deeplearning-iap-material/train_32x32.mat")
test_mat = keras.utils.get_file("test_32x32.mat", "https://s3-ap-southeast-1.amazonaws.com/deeplearning-iap-material/test_32x32.mat")
extra_mat = keras.utils.get_file("extra_32x32.mat", "http://ufldl.stanford.edu/housenumbers/extra_32x32.mat")
train_array = scipy.io.loadmat(train_mat)
train_array["X"].shape, train_array["y"].shape
test_array = scipy.io.loadmat(test_mat)
test_array["X"].shape, test_array["y"].shape
extra_array = scipy.io.loadmat(extra_mat)
extra_array["X"].shape, extra_array["y"].shape
print("Datasets loaded")

Datasets loaded


## Preprocessing Data

Training set and Extra set were combined together for training data.

Observations:
1. Using Extra set for training only with Training set for validation showed reduced accuracy compared to combined set and randomly setting validation set through sklearn function
2. Using combined set for traning resulted in higher accuracy than just using training set or extra set only, as expected

Crashes due to memory overflow occured in these scenarios:
1. ImageDataGenerator used on Combined/Extra set
2. Image augmentation through Scipy on any set




In [0]:
num_images = extra_array["X"].shape[-1] 
list_labels = extra_array["y"].reshape((num_images)) 
list_labels_train = train_array["y"].reshape((73257))
list_labels = np.append(list_labels , list_labels_train)
list_images = []

for i in range(num_images):
    image = extra_array["X"][:,:,:,i].reshape((32,32,3))
    list_images.append(image)
    
for i in range(73257):
    image = train_array["X"][:,:,:,i].reshape((32,32,3))
    list_images.append(image)
    
list_labels = np.asarray(list_labels, dtype='int32')
list_images = np.asarray(list_images)
print("Data formatted for Keras")

classes = list(set(list_labels))
num_classes = len(classes)
print("Classes set")

Data formatted for Keras
Classes set


## **Train-Validation Split Set**

Several split ratios were tested: 0.5, 0.2, 0.1, 0.05. For combined set, all ratios did not result in any significant changes (all around 95%) in accuracy except 0.05 which achieved 97% accuracy on average.

0.05 was chosen as combined set was already large enough, and 0.05 of the combined set should be able to provide enough variance of data compared to test set. 

In [0]:
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(list_images, list_labels, test_size=0.05)
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

# Build a CNN in Keras

Model was modifed to increase kernel size to 5, added in max normalisation for kernel constraint, and added extra dropout layers to regularise the model as it was observed to be overfitting before.

In [0]:
model = Sequential()

model.add(Conv2D(96, kernel_size=7, activation='relu', input_shape=(32,32,3), strides= 2, kernel_constraint=max_norm(4.), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides= 2))
model.add(Dropout(0.1))

model.add(Conv2D(128, kernel_size=7, activation='relu', strides= 1, kernel_constraint=max_norm(4.), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides= 2))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=7, activation='relu', strides= 1, kernel_constraint=max_norm(4.), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides= 2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(2048, activation='relu', kernel_constraint=max_norm(2.)))
model.add(Dropout(0.25))
model.add(Dense(2048, activation='relu', kernel_constraint=max_norm(2.)))
model.add(Dropout(0.5))
model.add(Dense(11, activation='softmax', kernel_constraint=max_norm(2.)))

## Optimizer and Compilation

Stochastic gradient descent optimiser with settings as given in the AutoAugment paper, except for the decay in learning rate. 

In [0]:
sgd = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 16, 16, 96)        14208     
_________________________________________________________________
batch_normalization_7 (Batch (None, 16, 16, 96)        384       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 7, 7, 96)          0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 7, 7, 96)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 128)         602240    
_________________________________________________________________
batch_normalization_8 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 3, 3, 128)         0         
__________

## Structure Visualisation

In [0]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
SVG(model_to_dot(model).create(prog='dot', format='svg'))

## Training Model w/o Generator


Please run it for around 15+ epochs to achieve 97% accuracy on average. Highest accuracy achieved was 97.44%.

In [0]:
#early = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1, mode='min', baseline=None, restore_best_weights=False)
print("Training Progress:")
model_log = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=[X_val, y_val], )
#model.fit_generator(datagen.flow(X_train, y_train, batch_size=128), steps_per_epoch=len(X_train) // 128, epochs=2,             
                 #  validation_data=(X_val, y_val),validation_steps=X_train.shape[0] // 128,)

Training Progress:
Train on 574168 samples, validate on 30220 samples
Epoch 1/10
574168/574168 [==============================] - 181s 316us/step - loss: 0.3652 - acc: 0.8847 - val_loss: 0.1632 - val_acc: 0.9503
Epoch 2/10
574168/574168 [==============================] - 177s 309us/step - loss: 0.1750 - acc: 0.9480 - val_loss: 0.1284 - val_acc: 0.9614
Epoch 3/10
574168/574168 [==============================] - 177s 309us/step - loss: 0.1455 - acc: 0.9573 - val_loss: 0.1132 - val_acc: 0.9672
Epoch 4/10
574168/574168 [==============================] - 179s 311us/step - loss: 0.1301 - acc: 0.9622 - val_loss: 0.1089 - val_acc: 0.9679
Epoch 5/10
574168/574168 [==============================] - 178s 310us/step - loss: 0.1197 - acc: 0.9652 - val_loss: 0.1075 - val_acc: 0.9689
Epoch 6/10
574168/574168 [==============================] - 178s 310us/step - loss: 0.1118 - acc: 0.9676 - val_loss: 0.1044 - val_acc: 0.9703
Epoch 7/10
574168/574168 [==============================] - 178s 310us/step - 

## Training Model w Generator
(Turned Off for final submission) Not using generator for final submission as it will crash on combined set and does not contribute significantly to the accuracy.

In [0]:
#early = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1, mode='min', baseline=None, restore_best_weights=False)
#datagen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    #rotation_range=20,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #horizontal_flip=True)  
#datagen.fit(X_train)
#print("Training Progress:")
#model_log = model.fit_generator(datagen.flow(X_train, y_train, batch_size=128), steps_per_epoch=len(X_train) // 128, epochs=2,             
                 #  validation_data=(X_val, y_val),validation_steps=X_train.shape[0] // 128,)

## Saving and Downloading Trained Model

Saving model

In [0]:
model.save("model.h5")

Downloading (Does not work sometimes due to server timeout, download through sidebar instead)

In [0]:
from google.colab import files
files.download('model.h5')

## Training History

In [0]:
plt.plot(model_log.history['acc'])
plt.plot(model_log.history['val_acc'])
plt.title('Accuracy (Higher Better)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(model_log.history['loss'])
plt.plot(model_log.history['val_loss'])
plt.title('Loss (Lower Better)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## Evaluation (Validation Set)

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools, pickle

classes = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]

y_val_index = np.argmax(y_val, axis=1) # Convert one-hot to index
y_pred = model.predict(X_val)
y_pred_class = np.argmax(y_pred,axis=1)
print(classification_report(y_val_index, y_pred_class, target_names=classes))

In [0]:
plt.style.use('seaborn-dark')
def plot_confusion_matrix(cm, labels,
                          normalize=True,
                          title='Confusion Matrix (Validation Set)',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    else:
        #print('Confusion matrix, without normalization')
        pass

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

plt.figure(figsize=(14,7))
cnf_matrix = confusion_matrix(y_val_index, y_pred_class)
cnf_matrix = confusion_matrix(y_val_index, y_pred_class)
plot_confusion_matrix(cnf_matrix, labels=classes)

# Running on the Test Set

In [1]:
# load Keras model from file

from keras.models import load_model

model = load_model("model.h5")

classes = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]

Using TensorFlow backend.


In [5]:
from sklearn.metrics import accuracy_score

test_array = scipy.io.loadmat(test_mat)
print(test_array.keys())

print("Test set has the following shape:")
test_array["X"].shape, test_array["y"].shape

num_images = test_array["y"].shape[0]

test_labels = test_array["y"].reshape((num_images))
test_images = []

for i in range(num_images):
    image = test_array["X"][:,:,:,i].reshape((32,32,3))
    test_images.append(image)
    
test_labels = np.asarray(test_labels, dtype='int32')
test_images = np.asarray(test_images)

print("test_images:", test_images.shape)

test_labels = to_categorical(test_labels)

test_labels_index = np.argmax(test_labels, axis=1)
test_preds = model.predict(test_images)
test_preds_class = np.argmax(test_preds,axis=1)
print("Test set accuracy score:", accuracy_score(test_labels_index, test_preds_class))

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])
Test set has the following shape:
test_images: (26032, 32, 32, 3)
Test set accuracy score: 0.9744161032575291


Submission by Pye Sone Kyaw for SUTD AI Challenge 2019
Last Modified 21/01/2019.